In [1]:
from transformers import pipeline

In [2]:
summarizer = pipeline("summarization")

In [3]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

print(summarizer(ARTICLE, max_length=130, min_length=30))

[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [4]:
sentence = "Hi David, would you mind sending over the forms we discussed? Thanks Joyce"

print(summarizer(sentence, max_length=10, min_length=2))

[{'summary_text': 'Hi David, would you mind sending'}]


In [6]:
examples = [
    'please sent the form to Brian',
    'could you ask operations whether Jose is on leave soon?',
    'Be sure to retain every request sent to us'
]

In [7]:
for ex in examples:
    summary = summarizer(ex, max_length=10, min_length=2)
    print(20*'=')
    print('SENTENCE:', ex)
    print()
    print('SUMMARY:', summary[0]['summary_text'])

Your max_length is set to 10, but you input_length is only 8. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


SENTENCE: please sent the form to Brian

SUMMARY: Brian has been asked to send in
SENTENCE: could you ask operations whether Jose is on leave soon?

SUMMARY: CNN.com will feature iRep
SENTENCE: Be sure to retain every request sent to us

SUMMARY: CNN will retain every request sent to


We need fine tuning FOR SURE.

We're able to use the pretrained summarizer from google T5 but it's 1.6 G.

In [9]:
from transformers import AutoModelWithLMHead, AutoTokenizer


model = AutoModelWithLMHead.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="pt", max_length=512)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
print(outputs)


tensor([[    0,   301, 13662,  1386,  3483,   235,     7,     6,   230,  6352,
             6,    19,  5008,   192,  4336, 12052,    13,    96,  1647,    49,
            53,     3,     9,  6136,  5009,    21,  9479,    16,     8,   166,
          1952,   121,   255,    19,  6141,    12,   341,    36,  4464,    12,
           662,  1076,     6,    11,    44,    80,    97,     6,   255,    47,
          4464,    12,  2641,  1076,    44,   728,     3,     5,     8,  5281,
             7,   130,   294,    13,    46, 10653, 13236,     3,     5]])


Named Entity recognition

In [10]:
nlp = pipeline("ner")

sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very" \
           "close to the Manhattan Bridge which is visible from the window."

print(nlp(sequence))


[{'word': 'Hu', 'score': 0.9995632767677307, 'entity': 'I-ORG', 'index': 1}, {'word': '##gging', 'score': 0.9915938973426819, 'entity': 'I-ORG', 'index': 2}, {'word': 'Face', 'score': 0.9982671737670898, 'entity': 'I-ORG', 'index': 3}, {'word': 'Inc', 'score': 0.9994403719902039, 'entity': 'I-ORG', 'index': 4}, {'word': 'New', 'score': 0.9994346499443054, 'entity': 'I-LOC', 'index': 11}, {'word': 'York', 'score': 0.9993270635604858, 'entity': 'I-LOC', 'index': 12}, {'word': 'City', 'score': 0.9993864893913269, 'entity': 'I-LOC', 'index': 13}, {'word': 'D', 'score': 0.9825621843338013, 'entity': 'I-LOC', 'index': 19}, {'word': '##UM', 'score': 0.936983048915863, 'entity': 'I-LOC', 'index': 20}, {'word': '##BO', 'score': 0.8987100720405579, 'entity': 'I-LOC', 'index': 21}, {'word': 'Manhattan', 'score': 0.9758240580558777, 'entity': 'I-LOC', 'index': 29}, {'word': 'Bridge', 'score': 0.9902493953704834, 'entity': 'I-LOC', 'index': 30}]


In [12]:
print(nlp("Rowan goes to school at UC Berkeley and he is in the MIDS program"))

[{'word': 'Rowan', 'score': 0.9989638924598694, 'entity': 'I-PER', 'index': 1}, {'word': 'UC', 'score': 0.9941582083702087, 'entity': 'I-ORG', 'index': 6}, {'word': 'Berkeley', 'score': 0.955276608467102, 'entity': 'I-ORG', 'index': 7}, {'word': 'MI', 'score': 0.6695720553398132, 'entity': 'I-MISC', 'index': 13}, {'word': '##DS', 'score': 0.7307611107826233, 'entity': 'I-MISC', 'index': 14}]


# Roberta Language Model

In [14]:
from transformers import RobertaConfig, RobertaModel

# Initializing a RoBERTa configuration
configuration = RobertaConfig()

# Initializing a model from the configuration
model = RobertaModel(configuration)

# Accessing the model configuration
configuration = model.config

In [15]:
configuration

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [22]:
from transformers import TFPreTrainedModel

ImportError: cannot import name 'TFPreTrainedModel' from 'transformers' (/Users/rowancassius/opt/anaconda3/lib/python3.7/site-packages/transformers/__init__.py)